In [1]:
import scanpy as sc
import anndata as ad
import scgen
import numpy as np
import torch
import random
import os
from icecream import ic
import gc

Global seed set to 0


In [2]:
def set_seed(seed):
    ic('Setting seed to', seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    

In [3]:
train = sc.read(
    '/data/Experiments/Benchmark/SCDISENTANGLE_REPRODUCE/Datasets/preprocessed_datasets/kang.h5ad'
    )

In [4]:
set_seed(1)

ic| 'Setting seed to', seed: 1


In [5]:
_train_adata = train[train.obs[f'split_stimulated_B'] == 'train'].copy()

In [6]:
# Compute train median
_sums = _train_adata.X.sum(axis=1, keepdims=True)
data_median = np.median(_sums)

# Normalize by train median
sc.pp.normalize_total(train, target_sum=data_median)
sc.pp.log1p(train)

In [7]:
train_new = train[~((train.obs["cell_type"] == "B") &
                    (train.obs["condition"] == "stimulated"))]

In [8]:
train_new = train_new.copy()

In [9]:
scgen.SCGEN.setup_anndata(train_new, batch_key="condition", labels_key="cell_type")

In [10]:
model = scgen.SCGEN(train_new)

In [11]:
model.train(
    max_epochs=100,
    batch_size=32,
    early_stopping=True,
    early_stopping_patience=25
)

/home/SE/miniconda3/envs/scgen/lib/python3.10/site-packages/pytorch_lightning/core/lightning.py:2054: DeprecationWarning: `torch.distributed._sharded_tensor` will be deprecated, use `torch.distributed._shard.sharded_tensor` instead
  from torch.distributed._sharded_tensor import pre_load_state_dict_hook, state_dict_hook
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 26/100:  26%|██▌       | 26/100 [01:24<03:59,  3.24s/it, loss=148, v_num=1]
Monitored metric elbo_validation did not improve in the last 25 records. Best score: 628.846. Signaling Trainer to stop.


In [14]:
pred, delta = model.predict(
    ctrl_key='control',
    stim_key='control',
    celltype_to_predict='B'
)
pred.obs['condition'] = 'pred'

/home/SE/miniconda3/envs/scgen/lib/python3.10/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/SE/miniconda3/envs/scgen/lib/python3.10/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/SE/miniconda3/envs/scgen/lib/python3.10/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/SE/miniconda3/envs/scgen/lib/python3.10/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/SE/miniconda3/envs/scgen/lib/python3.10/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/SE/miniconda3/envs/scgen/lib/python3.10/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


In [15]:
pred[pred.obs['split_stimulated_B'] == 'train'].X.max()

array(5.026708, dtype=float32)

In [16]:
X_hat = model.get_normalized_expression(train_new)

AttributeError: 'SCGEN' object has no attribute 'get_normalized_expression'